## Import Libraries

In [1]:
import pandas as pd

## Load Excel File

In [3]:
# Load the Excel file and read the sheet named "Bullying"
xls = pd.ExcelFile("datasets/country_bullying.xlsx")
df = xls.parse("Bullying")

## Extract Headers and Data

In [4]:
# Use the 7th row as headers
# Fill any merged header cells
header_row = df.iloc[6].ffill()  

# Start reading data from row 9
data = df.iloc[8:].copy()

# Set the cleaned headers
data.columns = header_row  

## Clean Empty Columns and Drop Rows without Country

In [5]:
# Remove columns that are completely empty
data = data.dropna(axis=1, how='all')

# Remove rows where 'Countries and areas' is missing
data = data[data['Countries and areas'].notna()]

## Remove Rows After 'Summary'

In [6]:
# If a row contains 'Summary' in the 'Countries and areas' column, drop that and everything after it
summary_index = data[data['Countries and areas'].astype(str).str.strip().str.lower() == 'summary'].index
if not summary_index.empty:
    data = data.loc[:summary_index[0] - 1]

## Clean and Deduplicate Column Names

In [7]:
# Remove newlines and whitespace from column names
cleaned_columns = [str(col).strip().replace('\n', ' ') for col in data.columns]

# Rename duplicate columns (e.g. 'Male', 'Male.1') to make them unique
new_columns = []
counts = {}
for col in cleaned_columns:
    base = col
    if base in counts:
        counts[base] += 1
        new_columns.append(f"{base}.{counts[base]}")
    else:
        counts[base] = 0
        new_columns.append(base)
data.columns = new_columns

## Remove 'Total', 'Male', 'Female' Columns with Any Blanks

In [8]:
# Drop any 'Total', 'Male', or 'Female' column that contains even one missing or blank value
for col in data.columns:
    base = col.split('.')[0]  # Handle deduplicated columns like 'Male.1'
    if base in ['Total', 'Male', 'Female']:
        col_series = data[col]
        is_na = col_series.isna().any()  # Check for NaN values
        is_blank = col_series.astype(str).str.strip().eq('').any()  # Check for blank strings
        if is_na or is_blank:
            data.drop(columns=[col], inplace=True)

## Add ASEAN Label Column

In [9]:
# Define ASEAN country list
asean_countries = {
    "Brunei Darussalam", "Cambodia", "Indonesia", "Lao People's Democratic Republic",
    "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Viet Nam"
}

# Create a new column that labels each row as 'ASEAN' or 'Non-ASEAN'
data['ASEAN_Label'] = data['Countries and areas'].apply(
    lambda x: "ASEAN" if str(x).strip() in asean_countries else "Non-ASEAN"
)

## Set Singapore's 'Male', 'Female', and 'Total' Values to 25

In [10]:
# Identify the row for Singapore
singapore_mask = data['Countries and areas'].astype(str).str.strip().str.lower() == 'singapore'

# For the first existing column that starts with 'Male', 'Female', or 'Total', set its value to 25
for col in ['Male', 'Female', 'Total']:
    matching_cols = [c for c in data.columns if c.startswith(col)]
    if matching_cols:
        data.loc[singapore_mask, matching_cols[0]] = 25

## Export Final Cleaned Data to CSV

In [13]:
# Save the cleaned data to a new CSV file
data.to_csv("filtered_bullying_perc_gender_region.csv", index=False)